In [1]:
import pickle
import tensorflow as tf
from tensorflow.keras import models

import os
import numpy as np
import time
import pandas as pd

2022-07-25 08:43:00.845177: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def load_data(filename):
    data = []
    gene_names = []
    lines = open(filename).readlines()
    sample_names = lines[0].replace('\n', '').split('\t')[1:]
    dx = 1

    for line in lines[dx:]:
        values = line.replace('\n', '').split('\t')
        gene = str.upper(values[0])
        gene_names.append(gene)
        data.append(values[1:])
    data = np.array(data, dtype='float32')
    data = np.transpose(data)

    return data, sample_names, gene_names

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [3]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/DeepDEP/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/DeepDEP/prediction/code
/home/wmbio/WORK/gitworking/DeepDEP


In [4]:
with tf.device('/cpu:0'):
#     model_name = "model_custom_full_0720"
    model_name = "model_custom_full_0720"
    model_saved = models.load_model("prediction/custom_model/%s.h5" % model_name, 
                                    custom_objects={"coeff_determination": coeff_determination})
# model_paper is the full 4-omics DeepDEP model used in the paper
# user can choose from single-omics, 2-omics, or full DeepDEP models from the
# /data/full_results_models_paper/models/ directory

2022-07-25 08:43:53.019816: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-25 08:43:53.020717: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-25 08:43:53.059606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-25 08:43:53.059863: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.635GHz coreCount: 14 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 178.84GiB/s
2022-07-25 08:43:53.059874: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-25 08:43:53.060895: I tensorflow/stream_executor/platform/de

* **Custom**

In [10]:
# load TCGA genomics data and gene fingerprints
data_mut, sample_names_mut, gene_names_mut = load_data(
    "preprocessing/DATA/2022-07-13/predict_mut_prediction.txt")
data_exp, sample_names_exp, gene_names_exp = load_data(
    "preprocessing/DATA/2022-07-13/predict_exp_prediction.txt")
data_cna, sample_names_cna, gene_names_cna = load_data(
    "preprocessing/DATA/2022-07-13/predict_cna_prediction.txt")
data_meth, sample_names_meth, gene_names_meth = load_data(
    "preprocessing/DATA/2022-07-13/predict_meth_prediction.txt")
data_fprint_DepOIs, gene_names_fprint, function_names_fprint = load_data(
    "preprocessing/DATA/2022-07-13/predict_fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")



Datasets successfully loaded.




In [16]:
data_mut.shape

(10, 4539)

* **Paper**

In [5]:
data_mut, sample_names_mut, gene_names_mut = load_data(
    "prediction/data/prediction_data/predict_mut_prediction.txt")
data_exp, sample_names_exp, gene_names_exp = load_data(
    "prediction/data/prediction_data/predict_exp_prediction.txt")
data_cna, sample_names_cna, gene_names_cna = load_data(
    "prediction/data/prediction_data/predict_cna_prediction.txt")
data_meth, sample_names_meth, gene_names_meth = load_data(
    "prediction/data/prediction_data/predict_meth_prediction.txt")
data_fprint_DepOIs, gene_names_fprint, function_names_fprint = load_data(
    "prediction/data/prediction_data/predict_fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")



Datasets successfully loaded.




In [11]:
batch_size = 64
first_to_predict = 10
# predict the first 10 samples for DEMO ONLY, for all samples please substitute 10 by data_mut_tcga.shape[0]
# prediction results of all 8238 TCGA samples can be found in /data/full_results_models_paper/predictions/

In [16]:
with tf.device('/cpu:0'):
    t = time.time()
    data_pred = np.zeros((first_to_predict, data_fprint_DepOIs.shape[0]))
    for z in np.arange(0, first_to_predict):
        data_pred_tmp = model_saved.predict([
            data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)
        data_pred[z] = np.transpose(data_pred_tmp)
        print("Unscreended sample %d predicted..." % z)

Unscreended sample 0 predicted...
Unscreended sample 1 predicted...
Unscreended sample 2 predicted...
Unscreended sample 3 predicted...
Unscreended sample 4 predicted...
Unscreended sample 5 predicted...
Unscreended sample 6 predicted...
Unscreended sample 7 predicted...
Unscreended sample 8 predicted...
Unscreended sample 9 predicted...


In [17]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:first_to_predict])

In [18]:
data_pred_df.to_csv('custom_mut_exp.csv')

In [ ]:
# pd.DataFrame.to_csv(data_pred_df, path_or_buf="/results/predictions/tcga_predicted_data_%s_demo.txt" % model_name, sep='\t', index_label='CRISPR_GENE', float_format='%.4f')
# print("\n\nPrediction completed in %.1f mins.\nResults saved in /results/predictions/tcga_predicted_data_%s_demo.txt\n\n" % (
#     (time.time()-t)/60, model_name))